## End to end model:Classifier

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf
import cv2
import os
from keras.utils import to_categorical

Using TensorFlow backend.


## 1 read feature and label

In [2]:
import h5py
import numpy as np

h5f = h5py.File('feature_label.h5','r')
features = h5f['features'][:]
labels = h5f['labels'][:]
h5f.close()

In [3]:
labels = labels.reshape([100,1])
print("du", features.shape)
print("du", labels.shape)

for i in range(100):
    labels[i] = int(i%2)

print("du", labels.shape)
labels = to_categorical(labels,2)
print("du", type(labels[0]))

du (100, 128, 128, 2048)
du (100, 1)
du (100, 1)
du <class 'numpy.ndarray'>


## 2 build classifier

In [4]:
with tf.device('/cpu:0'):
    input_tensor = Input((128, 128, 2048))

    # out 64 64 2048
    conv1_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv1_1')(input_tensor)
    conv1_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_2')(conv1_1)
    # out 32 32 2048
    conv2_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv2_1')(conv1_2)
    conv2_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_2')(conv2_1)
    # out 16 16 2048
    conv3_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv3_1')(conv2_2)
    conv3_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_2')(conv3_1)
    # out 8 8 2048
    conv4_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv4_1')(conv3_2)
    conv4_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_2')(conv4_1)
    # out 2048
    gap = GlobalMaxPooling2D()(conv4_2)

    # dropout
    p_out = Dropout(0.5)(gap)
    predictions = Dense(2, activation='softmax')(p_out)

    model = Model(inputs=input_tensor, outputs=predictions)
    model.summary()

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(optimizer='Adadelta', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
parallel_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 2048)    0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 64, 64, 4096)      75501568  
_________________________________________________________________
conv1_2 (Conv2D)             (None, 64, 64, 2048)      8390656   
_________________________________________________________________
conv2_1 (Conv2D)             (None, 32, 32, 4096)      75501568  
_________________________________________________________________
conv2_2 (Conv2D)             (None, 32, 32, 2048)      8390656   
_________________________________________________________________
conv3_1 (Conv2D)             (None, 16, 16, 4096)      75501568  
_________________________________________________________________
conv3_2 (Conv2D)             (None, 16, 16, 2048)      8390656   
__________

In [9]:
input_tensor = Input((128, 128, 2048))

# out 64 64 2048
conv1_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv1_1')(input_tensor)
conv1_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_2')(conv1_1)
# out 32 32 2048
conv2_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv2_1')(conv1_2)
conv2_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_2')(conv2_1)
# out 16 16 2048
conv3_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv3_1')(conv2_2)
conv3_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_2')(conv3_1)
# out 8 8 2048
conv4_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (2, 2),
                   activation='relu', padding='same', name='conv4_1')(conv3_2)
conv4_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_2')(conv4_1)
# out 2048
gap = GlobalMaxPooling2D()(conv4_2)

# dropout
p_out = Dropout(0.5)(gap)
predictions = Dense(2, activation='softmax')(p_out)

model = Model(inputs=input_tensor, outputs=predictions)
model.summary()

model.compile(optimizer='Adadelta', 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 2048)    0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 64, 64, 4096)      75501568  
_________________________________________________________________
conv1_2 (Conv2D)             (None, 64, 64, 2048)      8390656   
_________________________________________________________________
conv2_1 (Conv2D)             (None, 32, 32, 4096)      75501568  
_________________________________________________________________
conv2_2 (Conv2D)             (None, 32, 32, 2048)      8390656   
_________________________________________________________________
conv3_1 (Conv2D)             (None, 16, 16, 4096)      75501568  
_________________________________________________________________
conv3_2 (Conv2D)             (None, 16, 16, 2048)      8390656   
__________

In [ ]:
print("du", features.shape)
print("du", labels.shape)
parallel_model.fit(x=features, y=labels, batch_size=1, epochs=100, verbose=1)

du (100, 128, 128, 2048)
du (100, 2)
Epoch 1/100
100/100 [==============================] - 170s 2s/step - loss: 0.6940 - acc: 0.4000
Epoch 2/100
100/100 [==============================] - 170s 2s/step - loss: 0.6941 - acc: 0.4100
Epoch 3/100
 55/100 [===============>..............] - ETA: 1:15 - loss: 0.6941 - acc: 0.4545